<a href="https://colab.research.google.com/github/being-invincible/stockApp-Python/blob/main/STLF_vs_Prophet_Kats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Modules

In [ ]:
!pip3 install yfinance 
!pip install kats

     |████████████████████████████████| 6.4 MB 38.4 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 6.3 MB 26.2 MB/s 
     |████████████████████████████████| 934 kB 63.3 MB/s 
     |████████████████████████████████| 64 kB 3.3 MB/s 
     |████████████████████████████████| 31

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from plotly import graph_objs as go

import yfinance as yf
from datetime import date

# Scraping the Data from Yahoo Finance website



Specifying the Stock Ticker, Start and End date for scrapping data from yahoo finance. reset_index() is used to reset the scrapped data's first column which is date to the number of rows respectively.
You can comment reset_index() and view the changes via df.head()

In [ ]:
start = "2010-01-01"
today = date.today().strftime("%Y-%m-%d")

ticker = 'AAPL'

data = yf.download(ticker, start, today)
data = data.reset_index()
data.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.553025,493729600
1,2010-01-05,7.664286,7.699643,7.616071,7.656429,6.564355,601904800
2,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.459940,552160000
3,2010-01-07,7.562500,7.571429,7.466071,7.520714,6.447997,477131200
4,2010-01-08,7.510714,7.571429,7.466429,7.570714,6.490866,447610800


# Data Split and Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split

split_df = data.filter(['Date','Close'])

# Filling the missing dates and values in the time series df

#r = pd.date_range(start=split_df['Date'].min(), end=split_df['Date'].max())
#split_df = split_df.set_index('Date').reindex(r).rename_axis('time').reset_index()
#split_df = split_df.rename({'Close': 'value'}, axis='columns')

split_df = split_df.rename({'Date': 'time', 'Close': 'value'}, axis='columns')

train_df, test_df = train_test_split(split_df, test_size=0.2, shuffle=False)
print("Total Lenght of Data Frame :\t %d \nTrain df Length :\t\t %d \nTest df Lenght :\t\t %d \n"%(len(split_df), len(train_df), len(test_df)))


Total Lenght of Data Frame :	 3031 
Train df Length :		 2424 
Test df Lenght :		 607 



In [ ]:
train_df

,time,value
0,2010-01-04,7.643214
1,2010-01-05,7.656429
2,2010-01-06,7.534643
3,2010-01-07,7.520714
4,2010-01-08,7.570714
...,...,...
2419,2019-08-14,50.687500
2420,2019-08-15,50.435001
2421,2019-08-16,51.625000
2422,2019-08-19,52.587502


# Time Series Data Prep.

In [ ]:
# TimeSeriesData is the basic data structure in Kats to represented univariate and multivariate time series.
from kats.consts import TimeSeriesData

time_series_data = TimeSeriesData(train_df)

# If True, you have missing time series values
time_series_data.is_data_missing()

True

# STLF Model

This model starts from decomposing the time series data with STL decomposition
then it fits individual foreasting model on the de-seasonalized components.

It re-seasonalizes the forecasted results with seasonal data to produce the final
forecasting results.

In [ ]:
# import the param and model classes for Prophet model
from kats.models.stlf import STLFModel, STLFParams

# create a model param instance
params = STLFParams(method='prophet', m=12)

# create a prophet model instance
m = STLFModel(time_series_data, params)

# fit model simply by calling m.fit()
m.fit()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
# make prediction for next 30 month
forecast = m.predict(steps=len(test_df), include_history=False)

forecast

,time,fcst,fcst_lower,fcst_upper
0,2019-08-21,51.823951,49.039066,54.658241
1,2019-08-22,51.497522,48.746650,54.173606
2,2019-08-23,51.186644,48.344883,53.808099
3,2019-08-24,51.485701,48.742097,54.423164
4,2019-08-25,52.302357,49.615339,54.959362
...,...,...,...,...
602,2021-04-14,57.729204,44.771089,70.779140
603,2021-04-15,58.056895,44.042322,71.135940
604,2021-04-16,58.916956,45.116572,73.711247
605,2021-04-17,58.077316,44.837265,71.799745


# Error Testing

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

# Need to find null values' respective dates and remove the rows from both y_trues and y_preds
#y_trues = test_df['value']
#y_preds = forecast['fcst']

#mae = mean_absolute_error(y_trues, y_preds)
#mape = mean_absolute_percentage_error(y_trues, y_preds)

In [ ]:
# Plot testing

fig = go.Figure()
fig.add_trace(go.Scatter(x=test_df['time'], y=test_df['value'], name='Actual Stock Closing', line ={'color':'deepskyblue'}))
fig.add_trace(go.Scatter(x=forecast['time'], y=forecast['fcst'], name='Predicted Stock Closing', line ={'color':'limegreen'}))

fig.show()

In [ ]:
# Very poor results

# Prophet Model

In [ ]:
# import the param and model classes for Prophet model
from kats.models.prophet import ProphetModel, ProphetParams

# create a model param instance
params = ProphetParams(growth='linear', seasonality_mode='additive', cap= 10) # additive mode gives worse results

# create a prophet model instance
m = ProphetModel(time_series_data, params)

# fit model simply by calling m.fit()
m.fit()

# make prediction for next 30 month
forecast = m.predict(steps=len(test_df))

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
forecast

,time,fcst,fcst_lower,fcst_upper
0,2019-08-21,53.491969,50.838647,56.247510
1,2019-08-22,53.516330,50.684041,56.307107
2,2019-08-23,53.530736,50.744012,56.396576
3,2019-08-24,-89.251337,-92.130459,-86.385845
4,2019-08-25,-89.182651,-92.159844,-86.494099
...,...,...,...,...
602,2021-04-14,52.031317,49.162267,54.868092
603,2021-04-15,51.959942,49.000424,54.863631
604,2021-04-16,51.891593,48.942212,54.815886
605,2021-04-17,-95.734975,-98.712514,-92.814691


In [ ]:
# Plot testing

fig = go.Figure()
fig.add_trace(go.Scatter(x=test_df['time'], y=test_df['value'], name='Actual Stock Closing', line ={'color':'deepskyblue'}))
fig.add_trace(go.Scatter(x=forecast['time'], y=forecast['fcst'], name='Predicted Stock Closing', line ={'color':'limegreen'}))

fig.show()

In [ ]:
# Very Poor fit
# Still in development